<img src="img/mmcls-logo.png">

# MMClassification 2.0 Tutorials

<a href="" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MMClassification Repo**：[https://github.com/open-mmlab/mmclassification](https://github.com/open-mmlab/mmclassification)

**MMClassification 官方文档链接**：[https://mmclassification.readthedocs.io/en/1.x/index.html](https://mmclassification.readthedocs.io/en/1.x/index.html)

**MMClassification 2.0 教学视频**：Coming Soon~

**MMClassification 2.0 代码库介绍视频**：[https://www.bilibili.com/video/BV1x24y1R7mD](https://www.bilibili.com/video/BV1x24y1R7mD)

**加入微信社群方式**：关注公众号，选择 “加入我们” -> “微信社区”，即可获取入群二维码。非常期待你的到来呀~

**作者**：OpenMMLab

---

## Outline
0. Installation
1. Inference
2. Training
3. Testing
4. Training on Custom Datasets
5. Useful Tools

---

## 0. Installation
**Step 1. 安装 MIM**

**MIM**：MIM 为启动和安装 OpenMMLab 项目及其扩展以及管理 OpenMMLab 模型库提供了统一的接口。

In [ ]:
# !pip install openmim # 我们在conda环境中已经安装了，如果需要自己配置环境再安装

**Step 2. 安装依赖库 MMCV 和 MMEngine**

In [ ]:
# !mim install mmengine "mmcv>=2.0.0rc1" # 我们在conda环境中已经安装了，如果需要自己配置环境再安装

**Step 3. 安装 MMClassification**

**-e**：以可编辑模式安装项目。加上该参数后，如果对本地代码进行了修改，则代码可以在不重新安装代码库的情况下生效。

In [ ]:
!git clone https://gitee.com/open-mmlab/mmclassification.git -b 1.x
%cd mmclassification
# !pip install -e . # 我们在conda环境中已经安装了mmclassification，如果需要自己配置环境再安装，否则有可能会替换掉现有版本。事实上只要 cd 到 mmclassification 中，就可以 import 到 clone 下来的代码而不是安装在环境中的版本

**Step 4. 查看安装是否正确**

In [ ]:
import mmcls
print(mmcls.__version__)
print(mmcls.__file__)

---

## 1. Inference 
用 MMClassification [模型库](https://mmclassification.readthedocs.io/en/1.x/modelzoo_statistics.html#all-checkpoints)中预训练好的模型对图片进行预测。

**Step 1. 准备一张图片 `banana.png`**

<img src="img/banana.png">

**Step 2. 下载模型以及对应的配置文件**

```bash
mim download ${PACKAGE} ${CONFIG_FILENAME} --dest ${DESTINATION_PATH}
```

训练工具 `mim download` 参数讲解：

+ PACKAGE：工具包名
+ CONFIG_FILENAME：配置文件的文件名。**注意：不要加 `.py`**
+ DESTINATION_PATH：下载文件保存路径

更多请参考：[链接](https://openmim.readthedocs.io/en/latest/api.html#module-mim.commands.download)

In [ ]:
!mim download mmcls --config mobilenet-v2_8xb32_in1k --dest tutorial/

**Step 3. 用工具进行预测<font color = red>（没找到 mim 用于 inference）</font>**

```bash
python demo/image_demo.py \
    ${IMG_PATH} \
    ${CONFIG_FILE} \
    ${CHECKPOINT_PATH} \
    [--device ${YOUR_DEVICE}]
```

推理工具 [demo/image_demo.py](https://github.com/open-mmlab/mmclassification/blob/1.x/demo/image_demo.py) 参数讲解:
+ IMG_PATH：测试图片所在路径

+ CONFIG_FILE：配置文件所在路径

+ CHECKPOINT_PATH：模型文件所在路径

+ device：设备，默认 'cuda:0'

In [ ]:
# 在管理节点用 CPU执行
!python demo/image_demo.py demo/dog.jpg tutorial/mobilenet-v2_8xb32_in1k.py tutorial/mobilenet_v2_batch256_imagenet_20200708-3b2dc3af.pth --device cpu
# 在GPU节点执行
!srun -p gpu -n 1 --gres gpu:1 python demo/image_demo.py demo/dog.jpg tutorial/mobilenet-v2_8xb32_in1k.py tutorial/mobilenet_v2_batch256_imagenet_20200708-3b2dc3af.pth

加载模型、推理的代码实现再 `demo/image.py` 中，可以把对应的代码集成到自己的程序中，实现模型推理的功能

---

## 2. Training
用 MMClassification 在 CIFAR10 数据集上训练分类模型 ResNet-18

**Step 1. 准备数据集**

[CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) 数据集包含 10 个类别共 60000 张图片，每个类别有 6000 张图片，其中 5000 张作为训练图片，1000 张作为测试图片。

<img src="img/CIFAR.png">

我们推荐在 MMClassification 代码库的根目录下新建名为 `data` 的文件夹，并把数据集放在 `data` 文件夹里。

MMClassification 支持**自动下载** CIFAR10 数据集，我们只需要在配置文件里的 `data_root` 字段里设置好数据存放的路径，同时用 `test_mode=False` 或者 `test_mode=True` 来区分训练集和测试集即可。

```bash
train_dataloader = dict(
    ...
    # Training dataset configurations
    dataset=dict(
        type='CIFAR10',
        data_root='data/cifar10',
        test_mode=False,
        pipeline=...,
    )
)

val_dataloader = dict(
    ...
    # Validation dataset configurations
    dataset=dict(
        type='CIFAR10',
        data_root='data/cifar10',
        test_mode=True,
        pipeline=...,
    )
)

test_dataloader = val_dataloader
```

更多准备数据集的详情请参考：[链接](https://mmclassification.readthedocs.io/en/1.x/user_guides/dataset_prepare.html)

**Step 2. 下载 ResNet-18 模型的配置文件**

In [ ]:
!mim download mmcls --config resnet18_8xb16_cifar10 --dest tutorial/

**Step 3. 训练**

In [ ]:
# !mim train mmcls --launcher slurm --gpus 2 --gpus-per-node 2 --partition gpu configs/resnet/resnet18_8xb16_cifar10.py --auto-scale-lr
!mim train mmcls --launcher slurm -g 2 -G 2 -p gpu tutorial/resnet18_8xb16_cifar10.py --auto-scale-lr

**Step 4. 理解打印出来的 log 日志文件**
+ 系统环境
+ 配置文件
+ 训练过程中学习率的变化、loss 变化、消耗时间、消耗内存
+ 验证集上的准确率

---

## 3. Testing
用 MMClassification 在 CIFAR10 数据集上测试分类模型 ResNet-18

**Step 1. 准备数据集**

我们依旧使用 [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) 数据集来演示

**Step 2. 下载 ResNet-18 模型以及对应的配置文件**

我们直接使用上一节下载好的 ResNet-18 模型即可。

**Step 3. 用工具开始测试**

```bash
mim test ${PACKAGE} ${CONFIG_FILE} --checkpoint ${CHECKPOINT_FILE} --out ${OUT_FILE} --dump ${DUMP_FILE} --other_args ${OTHER_ARGS}
```

训练工具 `mim test` 参数讲解：

+ PACKAGE：工具包名
+ CONFIG_FILE：配置文件所在路径
+ CHECKPOINT_FILE：模型文件所在路径
+ OTHER_ARGS: 其他和各自工具包（代码库）相关的参数。比如 MMClassification 中的
    + OUT_FILE：保存模型在测试集上的整体评估结果文件所在路径。默认自动保存在 `work_dir` 文件夹下
    + DUMP_FILE：保存模型对每个样本的预测结果文件所在路径。默认不设置该参数，也就是不会保存该文件。如果要保存该文件，文件命名必须以 .pkl 为后缀


更多请参考：[链接](https://openmim.readthedocs.io/en/latest/api.html#module-mim.commands.test)

In [ ]:
!mim test mmcls --launcher slurm --gpus 2 --gpus-per-node 2 --partition gpu configs/resnet/resnet18_8xb16_cifar10.py --checkpoint tutorial/resnet18_b16x8_cifar10_20210528-bd6371c8.pth --dump tutorial/result_cifar.pkl

**解读保存下来的评估结果文件**

**自定义多 metrics 测试（跳过）**

除了 `Top-1 Accuracy`，MMClassification 还支持许多其他评估指标（详情请参考：[链接](https://mmclassification.readthedocs.io/en/1.x/advanced_guides/evaluation.html)），我们只需要修改配置文件中的 `val_evaluator` 和 `test_evaluator` 里的内容即可。

In [ ]:
!cp configs/resnet/resnet18_8xb16_cifar10.py configs/resnet/resnet18_8xb16_cifar10_4metrics.py

比如，我们要测试模型的 `Top-1 Accuracy`、 `Top-5 Accuracy`、 `precision` 和 `recall`，只需要将配置文件中 `val_evaluator` 和 `test_evaluator` 的内容修改成这样：

``` python
val_evaluator = [
  dict(type='Accuracy', topk=(1, 5)),
  dict(type='SingleLabelMetric', items=['precision', 'recall']),
]
test_evaluator = val_evaluator
```

In [ ]:
!mim test mmcls --launcher slurm --gpus 2 --gpus-per-node 2 --partition gpu configs/resnet/resnet18_8xb16_cifar10_4matrics.py --checkpoint tutorial/resnet18_b16x8_cifar10_20210528-bd6371c8.pth --dump tutorial/result_cifar_4metrics.pkl

---

## 4. Training on Custom Datasets
用 MMClassification 在自定义数据集上训练分类模型。

MMClassification 中支持自定义数据集的方法有 3 种：
1. 按照 `CustomDataset` 的数据格式整理数据集（[详情](https://mmclassification.readthedocs.io/en/1.x/user_guides/dataset_prepare.html#customdataset)）
2. 按照 `OpenMMLab 2.0 标注数据集` 的数据格式整理数据集（[详情](https://mmclassification.readthedocs.io/en/1.x/user_guides/dataset_prepare.html#openmmlab-2-0-standard-dataset)）
3. 自己实现一种数据集类型

我们在这里演示第 1 种：将数据整理成 `CustomDataset` 的格式。

在这种格式的数据集里，图片有多少个类别，就会有多少个子文件夹，子文件夹以`类名`命名，数据按类别存放在对应的子文件内。

比如：
<img src="img/CustomDataset_category.png">

**Step 1. 准备数据集**

我们这里使用的是 **同济子豪兄** 收集的 [水果分类数据集](https://zihao-openmmlab.obs.myhuaweicloud.com/20220716-mmclassification/dataset/fruit30/fruit30_split.zip)，该数据集有 30 个类别的图片。训练集里每个类别有 100+ 张图片，测试集里每个类别有 30+ 张图片。该数据集已经按 `CustomDataset` 的格式整理好了，直接使用即可。

我们在 MMClassification 代码库的根目录下新建名为 `data` 的文件夹，并把数据集下载放在 `data` 文件夹里。

In [ ]:
!mkdir data
!wget -P data https://zihao-openmmlab.obs.myhuaweicloud.com/20220716-mmclassification/dataset/fruit30/fruit30_split.zip

解压数据集

In [ ]:
!unzip -q data/fruit30_split.zip -d data

查看数据集的目录

In [ ]:
# !apt-get install tree # 如果环境中没有 tree 才需要安装
!tree -d data/fruit30_split

**Step 2. 下载模型以及对应的配置文件**

我们直接使用第 1 节下载好的 MobileNet v2 模型即可。

**Step 3. 修改配置文件**
+ 复制配置文件，并命名为 `mobilenet-v2_2xb32_fruit30.py`
+ 修改模型
    + 修改模型 `model` 里的类别数目 `num_classes`，设置为 30
    + 修改模型权值的加载路径 `load_from` ，设置为下载下来的 MobileNet v2 模型所在的路径
+ 修改数据
    + 将数据集类型 `dataset_type` 的值修改为 `CustomDataset`
    + 将训练加载器、验证加载器和测试加载器的数据集根目录 `data_root` 设置为 `data/fruit30_split`
    + 将训练加载器、验证加载器和测试加载器的数据集标注文件 `ann_file` 项删除，表示用文件夹名作为类名
+ 修改训练配置
    + 将 `train_cfg` 里的 `max_epochs` 改为 5
+ 修改优化器
    + 将 `optim_wrapper ` 里的 `lr` 改为原来的 1/8（线性缩放原则）
+ 修改杂项
    + 打印日志的间隔 `LoggerHook`
    + 保存模型文件的间隔 `CheckpointHook`

In [ ]:
!cp tutorial/mobilenet-v2_8xb32_in1k.py tutorial/mobilenet-v2_2xb32_fruit30.py

**Step 4. 用工具开始训练**

In [ ]:
!mim train mmcls --launcher slurm --gpus 2 --gpus-per-node 2 --partition gpu tutorial/mobilenet-v2_2xb32_fruit30.py --auto-scale-lr

**Step 5. 理解打印出来的 log 日志文件**
+ 系统环境
+ 配置文件
+ 检查模型初始化记录：`Loads checkpoint by local backend from path: .../mmclassification/mobilenet_v2_batch256_imagenet_20200708-3b2dc3af.pth`
+ 训练过程中学习率的变化、loss 变化、消耗时间、消耗内存
+ 验证集上的准确率

**Step 6. 测试训练好的模型**

In [ ]:
!python demo/image_demo.py banana.png tutorial/mobilenet-v2_2xb32_fruit30.py tutorial/mobilenet-v2_8xb32_fruit/epoch_5.pth --device cpu

---

## 5. Useful Tools

### 5.1 计算模型的参数量和计算量

**Step 1. 运行工具 `mim run`**

```bash
mim run ${PACKAGE} ${COMMAND} --yes ${YES} --other_args ${OTHER_ARGS}
```
参数讲解：
+ PACKAGE：工具包名
+ COMMAND：运行命令的名字
+ YES：不要求确认。bool类型的变量，默认为 True
+ OTHER_ARGS: 其他传入各工具包运行命令的参数

更多请参考：[链接](https://openmim.readthedocs.io/en/latest/api.html#module-mim.commands.run)

如果想查看某个工具包支持哪些命令以及使用方法，可使用命令：

```bash
mim run ${PACKAGE} -h
```

比如，查看 MMClassification 目前支持的命令：

In [ ]:
!mim run mmcls -h

**Step 2. 安装 `fvcore`**

In [ ]:
!pip install fvcore

**Step 3. 计算 Resnet-18 模型的参数量和计算量**

```bash
mim run mmcls get_flops ${CONFIG_FILE} --shape ${INPUT_SHAPE}
```
参数讲解：
+ CONFIG_FILE：配置文件所在路径
+ INPUT_SHAPE：设置输入形状。支持设置一个值或者两个值，比如 `--shape 256` 或者 `--shape 224 256`，如果不设置该参数，则默认输入形状设置为 `--shape 224 224`

详情请参考：[链接](https://mmclassification.readthedocs.io/en/latest/tools/analysis.html#get-the-flops-and-params-experimental)

In [ ]:
!mim run mmcls get_flops tutorial/resnet18_8xb16_cifar10.py

### 5.2 分析模型预测正确和错误的典型例子


```bash
mim run mmcls analyze_results ${CONFIG_FILE} ${RESULT} --out-dir ${OUT_DIR} --topk ${TOPK} --cfg-options ${CFG_OPTIONS}
```
参数讲解：
+ CONFIG_FILE：配置文件所在路径
+ RESULT：模型测试时保存的每个样本的预测结果文件（在 test 时，设置 `dump` 参数保存的文件），文件类型为 pkl 或者 json
+ OUT_DIR：保存模型预测正确和错误的典型例子图片的文件夹路径
+ TOPK：保存多少张模型预测正确和错误的典型例子。默认值为 20
+ CFG_OPTIONS：合入配置文件中的其他信息

详情请参考：[链接](https://mmclassification.readthedocs.io/en/latest/tools/analysis.html#view-typical-results)

**Step 1. 评估水果分类模型，并保存模型预测每个样本的结果文件**

In [ ]:
!mim test mmcls --launcher slurm --gpus 2 --gpus-per-node 2 --partition gpu tutorial/mobilenet-v2_2xb32_fruit30.py --checkpoint work_dirs/mobilenet-v2_2xb32_fruit30/epoch_5.pth --dump tutorial/result_fruit.pkl

**Step 2. 保存模型预测正确和错误的典型例子**

In [ ]:
!mkdir -p tutorial/typical_result
!mim run mmcls analyze_results tutorial/mobilenet-v2_2xb32_fruit30.py tutorial/result_fruit.pkl  --out-dir tutorial/typical_result

**Step 3. 分析错误的典型例子**

真实标签：猕猴桃

预测标签：梨

<img src="img/fail_example_2.png">

### 5.3 绘制混淆矩阵 <font color = red>（2.0暂时还不支持，可关注文档更新）</font>

MMCls 0.x 里该功能是在 mmcls.core.evaluation 里：https://mmclassification.readthedocs.io/en/latest/api/generated/mmcls.core.calculate_confusion_matrix.html#mmcls.core.calculate_confusion_matrix
但根据 1.0 里的迁移说明：https://mmclassification.readthedocs.io/en/1.x/migration.html#mmcls-core
该 mmcls.core.evaluation 是迁移至 mmcls.evaluation 里了
查阅文档，暂时还不支持

### 5.4 可视化数据增强流程

```bash
mim run mmcls browse_dataset ${CONFIG_FILE} \
                    [--output-dir ${OUTPUT_DIR}] \
                    [--phase ${DATASET_PHASE}] \
                    [--show-number ${NUMBER_IMAGES_DISPLAY}] \
                    [--show-interval ${SHOW_INTERRVAL}] \
                    [--mode ${DISPLAY_MODE}] \
                    [--rescale-factor ${RESCALE_FACTOR}] \
                    [--channel-order ${CHANNEL_ORDER}] \
                    [--cfg-options ${CFG_OPTIONS}]
```
参数讲解：
+ CONFIG_FILE：配置文件所在路径
+ OUTPUT_DIR：保存可视化图片的路径。如果不设置，则默认为 `''`，表示不保存
+ DATASET_PHASE：选择可视化数据集的训练集、验证集还是测试集。取值必须是 `['train', 'val', 'test']` 中的一个。如果不设置，默认是 `train`
+ NUMBER_IMAGES_DISPLAY：可视化图片的数量。如果不设置，默认显示数据集里所有的图片
+ SHOW_INTERRVAL：设置显示图片的间隔
+ DISPLAY_MODE：展示图片的模式。取值可以是 `['original', 'transformed', 'concat', 'pipeline']` 中的一个。如果不设置，默认是 `transformed`
+ RESCALE_FACTOR：图片尺寸缩放因子。如果图片本身过于太小或者小大，可以设置该参数，将图片可视化至合适的尺寸范围。
+ CHANNEL_ORDER：展示图片的通道顺序。取值可以是 `BGR` 或者是 `RGB`。如果没有指定，默认是 `BGR`
+ CFG_OPTIONS：合入配置文件中的其他信息

详情请参考：[链接](https://mmclassification.readthedocs.io/en/1.x/useful_tools/dataset_visualization.html#dataset-visualization)

**1. 可视化 CIFAR-10 数据集里的原图**

该数据集的图片尺寸很小，可以将 `rescale-factor` 参数设置大一些

mode：'original'

In [ ]:
!mkdir tutorial/cifar10_original_visualization
!im run mmcls browse_dataset tutorial/resnet18_8xb16_cifar10.py --output-dir 'tutorial/cifar10_original_visualization' --show-number 5 --mode 'original' --rescale-factor 10 --channel-order 'RGB'

**2. 并列可视化 CIFAR-10 数据集里的原图和按配置文件里数据加载流水线变换过后的图**

mode：'concat'

`resnet18_8xb16_cifar10.py` 配置文件里训练集的数据加载流水线处理如下：
```bash
pipeline=[
    dict(type='RandomCrop', crop_size=32, padding=4),
    dict(type='RandomFlip', prob=0.5, direction='horizontal'),
    dict(type='PackClsInputs')
]),
```
<img src='img/browseDataset_concat.png'></img>

In [ ]:
!mkdir tutorial/cifar10_concat_visualization
!mim run mmcls browse_dataset tutorial/resnet18_8xb16_cifar10.py --output-dir 'tutorial/cifar10_concat_visualization' --show-number 5 --mode 'concat' --rescale-factor 10 --channel-order 'RGB

**3. 可视化 CIFAR-10 数据集里按配置文件里数据加载流水线处理过程图**

mode：'pipeline'

`resnet18_8xb16_cifar10.py` 配置文件里训练集的数据加载流水线处理如下：
```bash
pipeline=[
    dict(type='RandomCrop', crop_size=32, padding=4),
    dict(type='RandomFlip', prob=0.5, direction='horizontal'),
    dict(type='PackClsInputs')
]),
```
<img src='img/browseDataset_pipeline.png'></img>

In [ ]:
!mkdir tutorial/cifar10_pipeline_visualization
!mim run mmcls browse_dataset tutorial/resnet18_8xb16_cifar10.py --output-dir 'tutorial/cifar10_pipeline_visualization' --show-number 5 --mode 'pipeline' --rescale-factor 10 --channel-order 'RGB'

### 5.5 可视化训练过程中的超参数变化图

在不进行训练的情况下，用可视化工具检查超参数（比如 `学习率 learning rate` 和 `动量 momentum`）是否按照自己的设想进行变化。

```bash
mim run mmcls vis_scheduler ${CONFIG_FILE} \
                    [-p, --parameter ${PARAMETER_NAME}] \
                    [-d, --dataset-size ${DATASET_SIZE}] \
                    [-n, --ngpus ${NUM_GPUs}] \
                    [-s, --save-path ${SAVE_PATH}] \
                    [--title ${TITLE}] \
                    [--style ${STYLE}] \
                    [--window-size ${WINDOW_SIZE}] \
                    [--cfg-options]
```
参数讲解：
+ CONFIG_FILE：配置文件所在路径
+ PARAMETER_NAME：检查的超参数名称。取值可以是 `['lr', 'momentum']` 中的一个。如果不设置，默认是 `lr`
+ DATASET_SIZE：数据集大小。**注意：默认是不用设置的，程序会自动按照配置文件里的设置构建数据集以获得数据集的大小，但是加载数据集是一个相对比较耗时的步骤。所以可以通过直接设置数据集大小来跳过构建数据集的过程，节省时间**
+ NUM_GPUs：用于训练的 GPU 数量，默认为 1
+ SAVE_PATH：可视化超参数变化图的保存路径
+ TITLE：可视化超参数变化图的标题，默认值是配置文件的名字
+ STYLE：plt 的风格，默认值是 `whitegrid`
+ WINDOW_SIZE：设置用于可视化窗格的大小，设置格式为 `'W*H'`。默认为 `'12*7'`
+ CFG_OPTIONS：合入配置文件中的其他信息

详情请参考：[链接](https://mmclassification.readthedocs.io/en/1.x/useful_tools/scheduler_visualization.html#)

**1. 可视化 `resnet18_8xb16_cifar10.py` 配置文件的学习率变化图**

<img src='img/lr_visualization.png'></img>

In [ ]:
!mim download mmcls --config resnet18_8xb16_cifar10 --dest tutorial/
!mim run mmcls vis_scheduler tutorial/resnet18_8xb16_cifar10.py --parameter 'lr' --ngpus 1 --save-path 'tutorial/resnet18_8xb16_cifar10_lr.png'

### 5.6 可视化类别激活图

目前，我们支持类别激活可视化的方法有：`GradCAM`、`GradCAM++`、`XGradCAM`、`EigenCAM`、`EigenGradCAM` 和 `LayerCAM`


```bash
mim run mmcls vis_cam ${IMG} ${CONFIG_FILE} ${CHECKPOINT} \
                [--target-layers ${TARGET-LAYERS}] \
                [--preview-model] \
                [--method ${METHOD}] \
                [--target-category ${TARGET-CATEGORY}] \
                [--save-path ${SAVE_PATH}] \
                [--vit-like] \
                [--num-extra-tokens ${NUM-EXTRA-TOKENS}]
                [--aug_smooth] \
                [--eigen_smooth] \
                [--device ${DEVICE}] \
                [--cfg-options ${CFG-OPTIONS}]
```
参数讲解：
+ IMG：要进行类别激活可视化的图片路径
+ CONFIG_FILE：配置文件所在路径
+ CHECKPOINT：模型文件所在路径
+ TARGET-LAYERS：获取哪一层或多层的激活图。如果不设置，默认为最后一个 block 的 norm layer。以 `ResNet-50` 为例：
    + 'backbone.layer4'：第四个残差层的输出
    + 'backbone.layer4.2'：第四个残差层里第三个 BottleNeck block 的输出
    + 'backbone.layer4.2.conv1'：上述 BottleNeck block 里 conv1 的输出
+ preview-model：是否打印模型里所有层的名字
+ METHOD：使用哪一个类别激活可视化的方法。默认使用 `GradCAM`
+ TARGET-CATEGORY：设置目标类别。如果不设置，默认使用网络预测的类别
+ SAVE_PATH：类别激活可视化图的保存路径
+ DEVICE：使用设备类型。默认为 `cpu`
+ CFG_OPTIONS：合入配置文件中的其他信息

更多参数请参考：[链接](https://mmclassification.readthedocs.io/en/1.x/useful_tools/cam_visualization.html)

**Step 1. 安装 `pytorch-grad-cam`**

In [ ]:
!pip install "grad-cam>=1.3.6"

**Step 2. 下载 ResNet-50 模型以及配置文件**

In [ ]:
!mim download mmcls --config resnet50_8xb32_in1k --dest tutorial/

**Step 3. 可视化示例图片 `demo/cat-dog.png` 的 `狗` 类别激活图**

在 ImageNet 里，238 对应的类别是：'Greater Swiss Mountain dog'

In [ ]:
!mim run mmcls vis_cam demo/cat-dog.png tutorial/resnet50_8xb32_in1k.py tutorial/resnet50_8xb32_in1k_20210831-ea4938fc.pth --target-layers 'backbone.layer4.2' --target-category 238 --save-path 'tutorial/dog_activation_map.png'

**Step 4. 可视化示例图片 `demo/cat-dog.png` 的 `猫` 类别激活图**

在 ImageNet 里，281 对应的类别是：'tabby, tabby cat'

In [ ]:
!mim run mmcls vis_cam demo/cat-dog.png tutorial/resnet50_8xb32_in1k.py tutorial/resnet50_8xb32_in1k_20210831-ea4938fc.pth --target-layers 'backbone.layer4.2' --target-category 281 --save-path 'tutorial/cat_activation_map.png'

---

## 6. 代码解析

MMClassification 和 MMEngine 之间的关系

<img src='img/MMCls 和 MMEngine 的关系 1.png'></img>
<img src='img/MMCls 和 MMEngine 的关系 2.png'></img>

MMEngine 详见：https://mmengine.readthedocs.io/zh_CN/latest/get_started/introduction.html

---
# 使用上交 slurm 提交作业的例子

** 写 run.sh 脚本提交作业 **

```shell
#!/bin/bash
# SBATCH -J train_MMCls
# SBATCH -p gpu
# SBATCH -N 1
# SBATCH --output = log.%j.out
# SBATCH --error = log.%j.err
# SBATCH -t 10:00:00
# SBATCH --gres = gpu:1
module load anaconda3/2019.07
source activate openmmlab
mim train mmcls resnet18_1xb128_cifar10_epoch5.py --launcher slurm --gpus 1 --gpus-per-node 1 --partition gpu
GPUS=1 GPUS_PER_NODE=1 ./tools/slurm_train.sh gpu train_MMCls resnet18_1xb128_cifar10_epoch5.py
```

** 在 base 环境下提交任务 **：

```shell
( mnist ) $ source deactivate
$ source activate base
( base ) sbatch run.sh
Submitted batch job 95835
```

**Step 5. 用工具开始训练**

```bash
[ENV_VARS] ./tools/slurm_train.sh \
        ${PARTITION} \
        ${JOB_NAME} \
        ${CONFIG_FILE} \
        ${WORK_DIR} \
        [PY_ARGS]
```

训练工具 [tools/slurm_train.sh](https://mmclassification.readthedocs.io/en/1.x/user_guides/train_test.html#multiple-machines-managed-with-slurm) 参数讲解:

+ PARTITION：分区
+ JOB_NAME：指定作业名称
+ CONFIG_FILE：配置文件所在路径
+ WORK_DIR：工作目录

`ENV_VARS` 配置 slurm 作业的环境变量：

+ GPUS：使用 GPU 的个数，默认使用 8 张 GPU
+ GPUS_PER_NODE：每个节点分配 GPU 的个数，默认每个节点分配 8 张 GPU
+ CPUS_PER_TASK：单个任务所需要的 CPU 核数，默认每个任务使用 5 个 GPU
+ SRUN_ARGS：其他 srun 命令的参数，详情请参考：[链接](https://slurm.schedmd.com/srun.html)